# Problem 1: (50% for CSC 446, 50% for CSC 546)

To ensure that you read and fully understand the template source code for discrete-event simulation of single-queue-single-server systerm, translate the sample Jave code posted in Brightspace--Content--Weeks 2,3 to Python. **Note that you are not allowed to use existing python function to generate exponential or normal variates.**

Using the given default parameters given in the sample code, generate 400 customers, and record their arrival times, departure times, service times, and waiting times in the queue in the following format:

| Customer #  | Arrival Time      |  Departure Time | Service Time for the Customer| The Customer's Waiting Time in the Queue|
|:---------------------:|:------------:|:-------------|:----------:|:-----|






## Import libraries

In [2]:
import random
import math
import pandas as pd

In [3]:
def normal(mean, sigma):
    global SaveNormal, NumNormals, PI
    # other global variables
    SaveNormal = 0
    NumNormals = 0
    PI = 3.1415927
    
    ReturnNormal = 0
    # should we generate two normals?
    if(NumNormals == 0):
        r1 = random.random()
        r2 = random.random()
        ReturnNormal = ((-2*math.log(r1))**0.5)*math.cos(2*PI*r2)
        SaveNormal = ((-2*math.log(r1))**0.5)*math.sin(2*PI*r2)
        NumNormals = 1
    else:
        NumNormals = 0
        ReturnNormal = SaveNormal
    
    return ReturnNormal*sigma + mean
    
    
def ScheduleDeparture():
    global QueueLength
    
    ServiceTime = normal(MeanServiceTime, SIGMA)
    while ServiceTime < 0:
        ServiceTime = normal(MeanServiceTime, SIGMA)
    
    st.append(ServiceTime)
    depart = [departure, Clock+ServiceTime]
    FutureEventList.append(depart)
    NumberInService = 1
    QueueLength -= 1
    
    
    
def ProcessArrival(evt):
    global QueueLength, MaxQueueLength
    
    saved_customer_list.append([evt[1]])
    
    Customers.append(evt)
    QueueLength += 1
    if( NumberInService == 0):
        ScheduleDeparture()
    else: 
        TotalBusy += (Clock - LastEventTime)  # server is busy

  # adjust max queue length statistics
    if (MaxQueueLength < QueueLength): 
        MaxQueueLength = QueueLength

  # schedule the next arrival
    next_arrival = [arrival, Clock+exponential(MeanInterArrivalTime)]
    FutureEventList.append( next_arrival )
    LastEventTime = Clock

    
def ProcessDeparture(evt):
    global SumResponseTime, TotalBusy, LastEventTime, NumberOfDepartures, LongService
    
    last_event = saved_customer_list[len(saved_customer_list)-1] 
    last_event += [evt[1]]
    
    if len(saved_customer_list) == 1:
        last_event += [st[len(st)-1], 0]
    else:
        prev = saved_customer_list[len(saved_customer_list)-2]
        if prev[1] <= last_event[0]:
            last_event.append(st[len(st)-1])
            last_event.append(0)
        else:
            last_event.append(st[len(st)-1])
            last_event.append(prev[1]-last_event[0])
        
    
    
    # get the customer description
    finished = Customers[0]
    Customers.pop(0)
    # if there are customers in the queue then schedule
    # the departure of the next one
    if( QueueLength > 0 ): 
        ScheduleDeparture()
    else:
        NumberInService = 0
    # measure the response time and add to the sum
    response = (Clock - finished[1])
    SumResponseTime += response
    if( response > 4.0 ): 
        LongService += 1 # record long service
    TotalBusy += (Clock - LastEventTime )
    NumberOfDepartures += 1
    LastEventTime = Clock

    
def exponential(mean):
    return -mean*math.log(random.random())
    
    
def initialization():
    evt = [arrival, exponential(MeanInterArrivalTime)]
    FutureEventList.append(evt)
        

def ReportGeneration():
    RHO = TotalBusy/Clock
    AVGR = SumResponseTime/TotalCustomers
    PC4 = (LongService)/TotalCustomers

    print("SINGLE SERVER QUEUE SIMULATION - GROCERY STORE CHECKOUT COUNTER ")
    print(f'\tMEAN INTERARRIVAL TIME {MeanInterArrivalTime}')
    print(f"\tMEAN SERVICE TIME {MeanServiceTime}")
    print(f"\tSTANDARD DEVIATION OF SERVICE TIMES {SIGMA}")
    print(f"\tNUMBER OF CUSTOMERS SERVED {TotalCustomers}")
    print() 
    print(f"\tSERVER UTILIZATION {RHO}")
    print(f"\tMAXIMUM LINE LENGTH {MaxQueueLength}")
    print(f"\tAVERAGE RESPONSE TIME {AVGR} MINUTES")
    print(f"\tPROPORTION WHO SPEND FOUR ")
    print(f"\tMINUTES OR MORE IN SYSTEM {PC4}")
    print(f"\tSIMULATION RUNLENGTH {Clock} MINUTES")
    print(f"\tNUMBER OF DEPARTURES {TotalCustomers}")


        
if __name__ == '__main__':
    # global variable
    global Clock, MeanInterArrivalTime, MeanServiceTime, SIGMA, LastEventTime, TotalBusy, MaxQueueLength, SumResponseTime
    global QueueLength, NumberInService, TotalCustomers, NumberOfDepartures, LongService
    Clock = 0.0
    MeanInterArrivalTime = 4.5
    MeanServiceTime = 3.2
    SIGMA = 0.6
    LastEventTime = 1000
    TotalBusy = 0
    MaxQueueLength = 0
    SumResponseTime = 0
    QueueLength = 0
    NumberInService = 0
    TotalCustomers = 400
    NumberOfDepartures = 0
    LongService = 0

    # initialize attributes/data
    random.seed(10)
    arrival = 1
    departure  = 2
    FutureEventList = []
    Customers = []
    saved_customer_list = [] 
    st = []
    
    initialization()
    
    while(NumberOfDepartures < TotalCustomers ):
        evt = FutureEventList[0] # get imminent event
        FutureEventList.pop(0)   # remove it from the eventlist
        Clock = evt[1]           # simulation time
        if evt[0] == arrival: 
            ProcessArrival(evt)
        else:
            ProcessDeparture(evt)

    ReportGeneration()
 

SINGLE SERVER QUEUE SIMULATION - GROCERY STORE CHECKOUT COUNTER 
	MEAN INTERARRIVAL TIME 4.5
	MEAN SERVICE TIME 3.2
	STANDARD DEVIATION OF SERVICE TIMES 0.6
	NUMBER OF CUSTOMERS SERVED 400

	SERVER UTILIZATION 0.42042593137234646
	MAXIMUM LINE LENGTH 0
	AVERAGE RESPONSE TIME 3.1570485200811866 MINUTES
	PROPORTION WHO SPEND FOUR 
	MINUTES OR MORE IN SYSTEM 0.085
	SIMULATION RUNLENGTH 1725.4050071078123 MINUTES
	NUMBER OF DEPARTURES 400


In [4]:
df = pd.DataFrame(saved_customer_list)
df.columns = ['arrival time', 'departure time', 'Service time', 'waiting time in queue']
df

,arrival time,departure time,Service time,waiting time in queue
0,2.518476,5.029866,2.511390,0.000000
1,9.625786,12.997841,3.372055,0.000000
2,11.540333,13.601775,2.061442,1.457509
3,16.559788,20.715276,4.155488,0.000000
4,16.575309,20.730467,4.155158,4.139967
...,...,...,...,...
395,1704.076095,1706.954410,2.878315,3.060282
396,1710.440294,1712.475023,2.034730,0.000000
397,1715.360071,1719.501093,4.141022,0.000000
398,1717.538503,1719.527337,1.988834,1.962590


# Problem 2 (50% for CSC 446, 25% for CSC 546)

1. Apply the Chi-square test to the recorded 400 customers in Problem 1 to test the hypothesis that the customer arrival
process follows a Poisson process, using the level of significance α = 0.05. 

Ans: Write your answer here.

$H_{0}:$ The Inter-arrival time is an exponential distribution

$H_{1}:$ The Inter-arrival time is not an exponential distribution

$\lambda = \frac{1}{meanIAT} = \frac{1}{4.5} = \frac{2}{9}$ where IAT means inter arrival time

Based on $400$ simulated samples:

Interval: from $\sqrt{400} = 20$ to $\frac{400}{5} = 80$ where $k = 20 $ and $k \leq \frac{400}{5}$

Let $k = 20$ such that probability of each interval is $p = \frac{1}{20} = 0.05$

CDF (which stands for cumulative distributive function) is $CDF = F(a_{i}) = 1 - e^{-\lambda a_{i}}$ and $F(a_{i}) = ip$

where $a_{i}$ is the end point of $i$th interval such that $i$ is between the interval $0\leq i \leq k$

Deriving $F(a_{i}) = 1 - e^{-\lambda a_{i}}$ and $F(a_{i}) = ip$ 

we get: $a_{i} = \frac{-1}{\lambda}\ln(1-ip)$ with $0 \leq i \leq k$, $\lambda = \frac{2}{9}$ and $p = 0.05$

Critical value $X_{0.05, 18}^2 = 28.869$ and $X_{0}^2 = 11.60$

Since $11.60 < 28.869$, the $H_{0}$ is accepted. Therefore arrival process is a Poisson Process 

In [7]:
import math

def inter_generate(i, lamb, p):
    return round(-1/lamb*math.log(1 - i*p), 3)

def chi_square_test(obs, exp, chi_sq_list):
    chi_sq = 0
    for i, j in zip(obs, exp):
        chi_sq += ((i - j)**2)/j
        chi_sq_list.append(chi_sq)
    return chi_sq

# initialization
lamb = 2/9
p = 0.05
interval = [0]
inter_arrival_time = [df.iloc[0, 0]]

for i in range(1, 20):
    interval.append(inter_generate(i, lamb, p))
interval.append(math.inf)

for i in range(1, 400):
    inter_arrival_time.append(df.iloc[i, 0] - df.iloc[i-1, 0])
    
# calculate frequency
freq = [0]*20
exp_freq = [400/20]*20
for i in range(400):
    for j in range(20):
        if interval[j] < inter_arrival_time[i] < interval[j+1]:
            freq[j] += 1
            break

table = {'Observed Frequency': freq, 'Expected Frequency': exp_freq}
chi_sq_list = []

chi_sq_val = chi_square_test(freq, exp_freq, chi_sq_list)
table['Chi_sq'] = chi_sq_list

print(f'chi_sq_val: {chi_sq_val:.6f}')

res = pd.DataFrame(table)
res

chi_sq_val: 11.600000


,Observed Frequency,Expected Frequency,Chi_sq
0,21,20.0,0.05
1,19,20.0,0.10
2,26,20.0,1.90
3,27,20.0,4.35
4,18,20.0,4.55
5,18,20.0,4.75
6,20,20.0,4.75
7,21,20.0,4.80
8,22,20.0,5.00
9,15,20.0,6.25


2. Apply the Chi-square test to the recorded 400 customers in Problem 1 to test the hypothesis that the customer depature
process follows a Poisson process, using the level of significance α = 0.05. 

Ans: Write your answer here. 

$H_{0}:$ The Inter-departure time is an exponential distribution

$H_{1}:$ The Inter-departure time is not an exponential distribution

$\lambda = \frac{1}{meanIDT} = \frac{1}{4.313512517769531} = 0.23183$ where IDT means inter departure time

Based on $400$ simulated samples:

Interval: from $\sqrt{400} = 20$ to $\frac{400}{5} = 80$ where $k = 20 $ and $k \leq \frac{400}{5}$

Let $k = 20$ such that probability of each interval is $p = \frac{1}{20} = 0.05$

$a_{i} = \frac{-1}{\lambda}\ln(1-ip)$ with $0 \leq i \leq k$, $\lambda = 0.23183$ and $p = 0.05$

Critical value $X_{0.05, 18}^2 = 28.869$ and $X_{0}^2 = 22$

Since $22 < 28.869$, the $H_{0}$ is accepted. Therefore arrival process is a Poisson Process 



In [6]:
import math

def inter_generate(i, lamb, p):
    return round(-1/lamb*math.log(1 - i*p), 3)

def chi_square_test(obs, exp, chi_sq_list):
    chi_sq = 0
    for i, j in zip(obs, exp):
        chi_sq += ((i - j)**2)/j
        chi_sq_list.append(chi_sq)
    return chi_sq

# initialization
lamb = 0.23183
p = 0.05
interval = [0]
idt = [df.iloc[0, 1]]

for i in range(1, 20):
    interval.append(inter_generate(i, lamb, p))
interval.append(math.inf)

for i in range(1, 400):
    idt.append(df.iloc[i, 1] - df.iloc[i-1, 1])
    
# calculate frequency
freq = [0]*20
exp_freq = [400/20]*20
for i in range(400):
    for j in range(20):
        if interval[j] < idt[i] < interval[j+1]:
            freq[j] += 1
            break

table = {'Observed Frequency': freq, 'Expected Frequency': exp_freq}
chi_sq_list = []

chi_sq_val = chi_square_test(freq, exp_freq, chi_sq_list)
table['Chi_sq'] = chi_sq_list

print(f'chi_sq_val: {chi_sq_val:.6f}')

res = pd.DataFrame(table)
res

chi_sq_val: 22.000000


,Observed Frequency,Expected Frequency,Chi_sq
0,9,20.0,6.05
1,14,20.0,7.85
2,14,20.0,9.65
3,13,20.0,12.10
4,19,20.0,12.15
5,19,20.0,12.20
6,17,20.0,12.65
7,23,20.0,13.10
8,22,20.0,13.30
9,18,20.0,13.50


3. What is your conclusion after the statistical test?

Ans: Write your answer here. 

**Hint: You can use inter-arrival times test for Poisson process: You can examine the inter-arrival times between successive events and check if they follow an exponential distribution. If the inter-arrival times follow an exponential distribution, then the arrival process can be considered a Poisson process.**

Based on question 1 and 2, the arrival process is a poisson process and departure process is a poisson process too. 

# End of Problem 2

# Problem 3 (CSC 546 only, 25% for CSC 546)

Change the service times to exponentially distributed with mean value of 3.2. Run your simulation to generate 400 customers, and and record their arrival times, departure times, service times, and waiting times using the same format as in Problem 1.


1. Apply the Chi-square test to the recorded customers to test the hypothesis that the customer arrival
process follows a Poisson process, using the level of significance α = 0.05. 

Ans: Write your answer here. 

2. Apply the Chi-square test to the recorded customers to test the hypothesis that the customer depature
process follows a Poisson process, using the level of significance α = 0.05. 

Ans: Write your answer here. 


3. What is your conclusion after the statistical test?

Ans: Write your answer here. 
**Hint: You can use inter-arrival times test for Poisson process: You can examine the inter-arrival times between successive events and check if they follow an exponential distribution. If the inter-arrival times follow an exponential distribution, then the arrival process can be considered a Poisson process.**

# End of Problem 3
